### Imports

In [3]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

import config
import pandas as pd
from src import data_ingestion
from src import data_providers
from src import benchmark
from src import portfolio
from src.symbols import Symbols

### Load Data

In [2]:
# --- Load Data ---
master_log = data_ingestion.create_master_log([
    config.INPUT_DATA_DIR / "us_mkt_transactions.csv",
    config.INPUT_DATA_DIR / "exus_mkt_transactions.csv"
]).set_index('Date')

master_log

Successfully loaded log: us_mkt_transactions.csv
Successfully loaded log: exus_mkt_transactions.csv


,Type,Symbol,Quantity,Price,Amount,Commission,Currency,Description,Exchange,Source
Date,,,,,,,,,,
2023-02-16,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,ID: 37fdafdc-d707-42fa-ba74-98b30cf9ab2a - DT2...,US Market,Sarwa Trade
2023-02-17,buy,PERI,0.407270,33.340000,-13.58,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-02-17,buy,TSLA,0.202666,201.020000,-40.74,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-02-17,buy,VOO,0.218967,372.070000,-81.47,NaN,USD,Trade Entry,NYSEArca,Sarwa Trade
2023-03-29,Net Dividend,VOO,NaN,NaN,0.24,NaN,USD,"Cash DIV @ 1.4874, Pos QTY: 0.218966527, Rec D...",NYSEArca,Sarwa Trade
...,...,...,...,...,...,...,...,...,...,...
2025-07-24,buy,BRK-B,0.124000,481.935484,-59.76,1.0,USD,Purchase of BRK-B,NYSE,Sarwa Trade - Interim
2025-07-24,buy,BTC-USD,0.000162,120615.384615,-19.60,NaN,USD,NaN,CCC,Sarwa Crypto
2025-07-24,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,Bank Deposit,US Market,Sarwa Trade - Interim


### `data_providers.py` Test

In [3]:
# --- 1. Initial Symbol Assessment ---
unique_symbols = master_log['Symbol'].dropna().unique()
found_df, missing_list = data_providers.assess_symbols(unique_symbols, master_log)

In [4]:
print("✅ The following symbols were automatically identified using yfinance:")
found_df

✅ The following symbols were automatically identified using yfinance:


,Name,Type,Exchange,Currency,Industry,Sector,DataProvider
Symbol,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,yfinance
...,...,...,...,...,...,...,...
JPMB,JPMorgan USD Emerging Markets Sovereign Bond ETF,etf,NYSEArca,USD,None,None,yfinance
SMOT,VanEck Morningstar SMID Moat ETF,etf,Cboe US,USD,None,None,yfinance
XLV,The Health Care Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,yfinance


In [5]:
print("❌ The following symbols were not found on yfinance:")
missing_list

❌ The following symbols were not found on yfinance:


['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY']

In [6]:
# --- 2. User Verification Step ---
# In this cell, the user reviews the table above and lists any symbols
# that yfinance identified incorrectly.
incorrectly_identified_symbols = [
    "AMR", "ASM"
]

# --- 3. Update Cache Based on User Input ---
data_providers.mark_symbols_as_user_defined(incorrectly_identified_symbols, master_log)

Updating cache for incorrectly identified symbols: ['AMR', 'ASM']
Cache updated successfully.


In [7]:
# --- 4. Get Final Lists for Data Fetching ---
# Reload the cache to get the final, corrected state
final_cache = data_providers._load_json_cache(config.METADATA_CACHE)

symbols_from_yfinance = [s for s, d in final_cache.items() if d.get('DataProvider') == 'yfinance']
symbols_from_user = [s for s, d in final_cache.items() if d.get('DataProvider') in ['missing', 'user_defined']]

print(f"Proceeding to fetch price data from yfinance for:\n{symbols_from_yfinance}")
print(f"Expecting user-provided price data & metadata for:\n{symbols_from_user}")

Proceeding to fetch price data from yfinance for:
['PERI', 'TSLA', 'VOO', 'UAE', 'XLK', 'BTC-USD', 'VOOG', 'NOBL', 'SPY', 'VHT', 'NVDA', 'SHOP', 'LUX', 'META', 'MSFT', 'IBB', 'SPHY', 'STIP', 'USRT', 'BA', 'ADBE', 'BKNG', 'CAT', 'EL', 'DE', 'COIN', 'OCSL', 'TPVG', 'GSBD', 'LLY', 'NVO', 'CLM', 'CEFS', 'GOF', 'SPXL', 'TQQQ', 'BRW', 'MOAT', 'AMZN', 'INDA', 'KSA', 'SPSM', 'VBK', 'IJR', 'PAK', 'MCHI', 'EWJ', 'MBB', 'ARKK', 'IPO', 'VXUS', 'XLF', 'ARM', 'ROM', 'RYAAY', 'APO', 'KKR', 'JPMB', 'SMOT', 'XLV', 'BX', 'BRK-B']
Expecting user-provided price data & metadata for:
['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY', 'ASM', 'AMR']


In [8]:
# --- 5. Complete Symbols DataFrame ---
symbol_df = data_providers.complete_symbol_df()
symbol_df

User-defined metadata file found. Merging with yfinance data...

Successfully created unified symbols DataFrame.


,Name,Exchange,Currency,Type,DataProvider,Industry,Sector
Symbol,,,,,,,
PERI,Perion Network Ltd.,NasdaqGS,USD,equity,yfinance,Internet Content & Information,Communication Services
TSLA,"Tesla, Inc.",NasdaqGS,USD,equity,yfinance,Auto Manufacturers,Consumer Cyclical
VOO,Vanguard S&P 500 ETF,NYSEArca,USD,etf,yfinance,None,None
UAE,iShares MSCI UAE ETF,NasdaqGM,USD,etf,yfinance,None,None
XLK,The Technology Select Sector SPDR Fund,NYSEArca,USD,etf,yfinance,None,None
...,...,...,...,...,...,...,...
IHC,International Holding Company PJSC,ADX,AED,equity,user_defined,Industrials,Conglomerates
EMAAR,Emaar Properties PJSC,DFM,AED,equity,user_defined,Real Estate - Development,Real Estate
MULTIPLY,None,ADX,AED,equity,user_defined,Financials,Asset Management


### Project Variables

In [ ]:
# --- Initialize Project Variables ---
start_date, end_date, date_range, last_market_day = config.project_dates(master_log.index)

# User-defined constants are accessed directly
benchmark_index = config.BENCHMARK_INDEX
tax_rate = config.TAX_RATE

In [ ]:
data_providers.yf_hist("STIP", start_date, last_market_day)

### `portfolio.py` Test

In [ ]:
# In your main script or notebook (e.g., src/portfolio.py)

# Get a unique list of every stock and its designated market from the log
symbol_market_pairs = master_log[['Symbol', 'Exchange']].dropna().drop_duplicates()

# Initialize empty DataFrames for price and split data
raw_splits = pd.DataFrame(index=date_range, columns=symbols)
holdings['price'] = pd.DataFrame(index=date_range, columns=symbols)

# Loop through the unique pairs to fetch data using the correct provider
for _, row in symbol_market_pairs.iterrows():
    symbol = row['Symbol']
    market = row['Exchange']

    print(f"Fetching data for {symbol} on {market}...")
    hist = None

    if market == 'US Market':
        # Use yfinance for all US market transactions
        hist = data_providers.yfinance_hist(symbol, start_date, end_date, last_market_day)
    else:
        # Use Twelve Data for all other markets
        # The 'Exchange' value (e.g., "ADX", "DFM") is used as the exchange code
        hist = data_providers.get_twelve_data_history(config.API_KEY_TWELVE_DATA, symbol, exchange=market)

    # Populate the main dataframes with the fetched data
    if hist is not None and not hist.empty:
        # Use .reindex() to align the data with your full date_range
        holdings['price'][symbol] = hist['Close'].reindex(date_range).ffill()
        raw_splits[symbol] = hist['Stock Splits'].reindex(date_range)

# Fill any remaining NaNs after fetching data for all symbols
raw_splits = raw_splits.fillna(0.0)

# Now, you can proceed with your existing holdings calculation loop
# ...